In [9]:
import requests
import numpy as np
import pandas as pd

def load_data(file_path, sep):
    # Read the local CSV file with specified separator
    df = pd.read_csv(file_path, sep=sep, parse_dates=['timestamp'], index_col='timestamp')
    return df

# Specify the local file path
file_path = 'Historical_Solar_Dataset.csv'

# Read dataset from the local file
PV_data = load_data(file_path, ',')

# Print the DataFrame to verify
print(PV_data)


                     Active_Power
timestamp                        
2017-01-20 00:00:00           0.0
2017-01-20 00:05:00           0.0
2017-01-20 00:10:00           0.0
2017-01-20 00:15:00           0.0
2017-01-20 00:20:00           0.0
...                           ...
2025-01-20 05:20:00           0.0
2025-01-20 05:25:00           0.0
2025-01-20 05:30:00           0.0
2025-01-20 05:35:00           0.0
2025-01-20 05:40:00           0.0

[827581 rows x 1 columns]


In [10]:
#extract hourly data
PV_data = PV_data[PV_data.index.minute == 0]
PV_data

,Active_Power
timestamp,
2017-01-20 00:00:00,0.0
2017-01-20 01:00:00,0.0
2017-01-20 02:00:00,0.0
2017-01-20 03:00:00,0.0
2017-01-20 04:00:00,0.0
...,...
2025-01-20 01:00:00,0.0
2025-01-20 02:00:00,0.0
2025-01-20 03:00:00,0.0


In [11]:
#Extract Weather Historical Data from OpenMeteo API

In [12]:
Predictors= ['temperature_2m', 'relativehumidity_2m', 'direct_radiation', 'diffuse_radiation',  'windspeed_10m', 'cloudcover']


weather_df = pd.DataFrame(columns = Predictors)

#latitude and longitude of the panel
lat = -23.760363
long = 133.874719

#start and end date of the available solar power data
start = str(PV_data.index[0].strftime('%Y-%m-%d'))
end = str(PV_data.index[-1].strftime('%Y-%m-%d'))

#historical weather for the panel
r = requests.get('https://archive-api.open-meteo.com/v1/archive', params={'latitude':lat, 'longitude': long, 'timezone': 'auto', 'start_date':start , 'end_date': end , 'hourly' : Predictors}).json() #timezone = auto so that it matches the local timezone

#create weather dataset
time = pd.to_datetime(np.array(r['hourly']['time']))
weather_df['timestamp'] = time

for p in Predictors:
        weather_df[p] = np.array(r['hourly'][p])

#set timestamp index
weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'])
weather_df.set_index('timestamp', inplace=True)

weather_df.head()

,temperature_2m,relativehumidity_2m,direct_radiation,diffuse_radiation,windspeed_10m,cloudcover
timestamp,,,,,,
2017-01-20 00:00:00,26.8,72,0.0,0.0,21.3,0
2017-01-20 01:00:00,26.2,75,0.0,0.0,20.9,6
2017-01-20 02:00:00,25.7,78,0.0,0.0,20.4,5
2017-01-20 03:00:00,25.3,80,0.0,0.0,19.1,12
2017-01-20 04:00:00,25.0,81,0.0,0.0,19.1,18


In [13]:
#Merge datasets and export

In [14]:
# Reset the index of PV_data to make 'timestamp' a column
PV_data_reset = PV_data.reset_index()

# Merge datasets using the 'timestamp' column
full_df_weather = PV_data_reset.merge(weather_df, how='left', left_on='timestamp', right_index=True)

# Check the merged DataFrame
print(full_df_weather.info())

# Save the merged dataset to CSV
full_df_weather.to_csv('Merged_SolarDataset.csv', sep='\t', index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68972 entries, 0 to 68971
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   timestamp            68972 non-null  datetime64[ns]
 1   Active_Power         67092 non-null  float64       
 2   temperature_2m       68972 non-null  float64       
 3   relativehumidity_2m  68972 non-null  int64         
 4   direct_radiation     68972 non-null  float64       
 5   diffuse_radiation    68972 non-null  float64       
 6   windspeed_10m        68972 non-null  float64       
 7   cloudcover           68972 non-null  int64         
dtypes: datetime64[ns](1), float64(5), int64(2)
memory usage: 4.2 MB
None
